# Blackbody Spectra Interactives

Welcome to the Blackbody Spectra interactives.  These interactive figures allow you to explore the properties of the spectra emitted by blackbodies (which are a good approximation for stars).  

A blackbody is an object that absorbs all light that hits it (which would make it appear 'black' if it didn't also emit light).   Though stars are not perfect absorbers of light, this assumption is very close to the truth.  This allows us to use a blackbody model to simulate the light given off by stars and estimate their temperature with reasonable accuracy.  

In [ ]:
# Author: Andrew Louwagie Gordon
# Date Created: 29May2018
# Last Modified: 02Mar2021
#
# Hacked on by Juan Cabanela to get color index calculations working.
# Juan Cabanela was the source of the data and code to interpret the data of real stellar blackbodies.
#
# Fixes attempted in March 2019 to get the color index right.  Turns out the flux to magnitude calibration needs
# to be done on a per bandpass basis.
#
# Fixes made in March 2021 to get it working under a more modern bqplot environment. Specifically I had to 
# purge attempts to plot arrays which had np.float128 values.

In [ ]:
# Import the necessary packages
from IPython.display import display
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter
from scipy.optimize import curve_fit

import ipywidgets as widgets
import bqplot as bq
import numpy as np
import pythreejs as p3j
import pandas as pd

import tempNcolor as tc
import number_formatting as nf
import starlib as star

In [ ]:
# Functions Definitions Block
# Define the constants
c = 3e8
h = 6.626e-34
k = 1.38e-23
R_Sun = 6.957e8     # Solar radius in meters
d_10pc = 3.086e+17  # 10 pc in meters

MSUM = True    # Set this to True to run a simpler interface for Interactive #2

# Define the initial shift in magnitudes
delta_V = None

def Wein(T):
    """
    This is Wein's Law and returns the peak wavelength.
    
    Parameters
    ----------
    T : float
        Temperature of star in Kelvin.

    Returns
    -------
    lamda_max : float
        The wavelength at which the blackboy spectrum peaks.
    
    """
    lamda_max = 0.002897755 / T
    return lamda_max

def blackbody(lamda, T):
    global c, h, k
    """
    This function takes the array of wavelengths and the temperature from the slider and returns an array of fluxes that
    correspond to the blackbody curve.
    
    Parameters
    ----------
    lamda: float or array of floats
        The wavelengths or set of wavelengths the spectrum goes through.
    T : float
        Temperature of star in Kelvin.

    Returns
    -------
    blackbody : float or array of floats
       The blackbody spectrum as modeled by Planck.
    
    """
    return np.array(( (2 * h * (c ** 2)) / (lamda ** 5) ) / ( np.exp((h * c) / (lamda * k * T)) - 1 ), dtype=float)

def cr(change=None):
    """
    This function updates the first figure    
    """
    global wavelengths
    spectrum = blackbody(wavelengths, Temp.value)
    
    pw = Wein(Temp.value)
    pwlist = nf.exp2LaTeX(pw) # Get LaTeX form of number
    peak_wavelength.value =  '{}'.format(pwlist[2]) # Write HTML form
        
    Model_Star.y = [blackbody(wavelengths,Temp.value)]
    
    if plotted_star.index == 0:
        star_fig.marks = [Proxima_Centauri, Model_Star]
        etemp = 3300
    elif plotted_star.index == 1:
        star_fig.marks = [Sun, Model_Star]
        etemp = 5800
    elif plotted_star.index == 2:
        star_fig.marks = [Polaris, Model_Star]
        etemp = 7200
    elif plotted_star.index == 3:
        star_fig.marks = [Alpha_Andromedae, Model_Star]
        etemp = 13000
    elif plotted_star.index == 4:
        star_fig.marks = [Eta_Ursae_Majoris, Model_Star]
        etemp = 16900
    elif plotted_star.index == 5:
        star_fig.marks = [Bellatrix, Model_Star]
        etemp = 22000
    
    star_pw_f = blackbody(Wein(etemp), etemp)
    pw_f = blackbody(Wein(Temp.value),Temp.value)
    
    if star_pw_f >= pw_f:
        y_star.max = star_pw_f
    else:
        y_star.max = pw_f
        
    if Temp.value >= 5100:
        fig.legend_location = 'top-right'
        star_fig.legend_location = 'top-right'
    else:
        fig.legend_location = 'top-left'
        star_fig.legend_location = 'top-left'
        

def cr2(change=None):
    """
    This function updates the second interactive figure.  
    """
    global wavelengths, U_spec, B_spec, V_spec, R_spec, I_spec
    
    # Update the blackbody spectrum
    spectrum = blackbody(wavelengths, Temp2.value)
    Blackbody.y = [spectrum]

    counts = tc.temp2rgb(Temp2.value) # Convert temperature into rgb counts
    hex_color = tc.rgb2hex(counts) # Convert the rgb counts into a hexidecimal value that reflects the color of the star
    star.StarMeshColor(star1, hex_color[0])
            
    # Update the flux magnitudes and color values
    (U_mag, B_mag, V_mag, R_mag, I_mag, U_flux, B_flux, V_flux, R_flux, I_flux) = compute_flux_and_mag(wavelengths, spectrum, debug=False) 
    
    # Update the filter curve lines to fit the updated blackbody
    U_line.y = [U_spec]
    B_line.y = [B_spec]
    V_line.y = [V_spec]
    R_line.y = [R_spec]
    I_line.y = [I_spec]
    
    # Update reported magnitudes
    U_mag_report.value = "{0:.2f}".format(U_mag)
    B_mag_report.value = "{0:.2f}".format(B_mag)
    V_mag_report.value = "{0:.2f}".format(V_mag)
    R_mag_report.value = "{0:.2f}".format(R_mag)
    I_mag_report.value = "{0:.2f}".format(I_mag)
    
    # Update reported colors
    UB_color = U_mag - B_mag
    BV_color = B_mag - V_mag
    VR_color = B_mag - R_mag
    RI_color = R_mag - I_mag
    UB_color_report.value = "{0:.2f}".format(UB_color)
    BV_color_report.value = "{0:.2f}".format(BV_color)
    VR_color_report.value = "{0:.2f}".format(VR_color)
    RI_color_report.value = "{0:.2f}".format(RI_color)
    
    # Update the numerical fluxes for each filter
    U_flux_report.value = str('{}'.format(nf.exp2LaTeX(U_flux,3)[2]))
    B_flux_report.value = str('{}'.format(nf.exp2LaTeX(B_flux,3)[2]))
    V_flux_report.value = str('{}'.format(nf.exp2LaTeX(V_flux,3)[2]))
    R_flux_report.value = str('{}'.format(nf.exp2LaTeX(R_flux,3)[2]))
    I_flux_report.value = str('{}'.format(nf.exp2LaTeX(I_flux,3)[2]))
    
def fini(change=None):
    """
    This function updates the second figure which shows the peak of the function with respect to the visible spectrum.
    """
    global my_wl, wide_line
    my_f = blackbody(wavelengths, Temp.value) 
    y_zeros = np.zeros_like(wavelengths)
    y_array = np.array([y_zeros, my_f])
    fin_y_array = y_array.transpose()    
    wide_line.y = [fin_y_array]   
    
def new_star(change):
    """
    Determines the name and appropriate data for new star and displays it.  Updates the final figure.
    Function written by Juan Cabanela and hacked by Andrew Louwagie Gordon to update the blackbody.
    """
    global spectra, fig, ind_text
    
    # Determine new index
    new_index = spec_data[spec_data['Name'] == starlist.value].index[0]
    
    # Set the displayed index value
    ind_text.value = str(new_index)
    
    # Update the data
    (lam, flux) = get_spectral_data(new_index)
    
    # Estimate sigma and then fit the Planck spectrum (also rescales flux)
    sigma_est = 2*np.std(flux-savgol_filter(flux, window, order))
    (flux, best_fit) = EstTempAndRescale(lam, flux, sigma)

    # Update plots after rescaling
    spectra.x = lam
    spectra.y = flux
    planck_spectra.x = spectra.x
    planck_spectra.y = best_fit

    # Update the plot
    fig.title = get_title(new_index)
    
    # Reset to not display model temperature
    planck_temp.value = False
    
    # Process the new blackbody too
    new_blackbody(change)

def new_blackbody(change):
    """
    Added by Andrew Louwagie Gordon (split into separate function by Juan Cabanela)
    to update the blackbody.
    """
    # Update the blackbody
    Model_Star_Vis.y = blackbody(lam, Temp3.value)
    
    if planck_temp.value == False:
        full_top.children = [starlist]
    elif planck_temp.value == True:
        full_top.children = [starlist, temp_text]

In [ ]:
# The following functions were developed by Juan Cabanela
def create_functions(debug=False):
    global Uspline, Bspline, Vspline, Rspline, Ispline

    ###
    ### The U, B, V, R filter transmission data was downloaded from the SNCosmos source repository at
    ### https://github.com/sncosmo/sncosmo/tree/master/sncosmo/data/bandpasses/bessell
    ### which are based on the work of Michael Bessell (Bessell, M.S. 1990, PASP, 102, 1181)
    ### to reproduce the classic Johnson-Cousins passbands using cheap optical glass filters.
    ###

    U = {300.0:0.000, 305.0:0.016, 310.0:0.068, 315.0:0.167, 320.0:0.287,
         325.0:0.423, 330.0:0.560, 335.0:0.673, 340.0:0.772, 345.0:0.841,
         350.0:0.905, 355.0:0.943, 360.0:0.981, 365.0:0.993, 370.0:1.000,
         375.0:0.989, 380.0:0.916, 385.0:0.804, 390.0:0.625, 395.0:0.423,
         400.0:0.238, 405.0:0.114, 410.0:0.051, 415.0:0.019, 420.0:0.000}

    B = {360.0:0.000, 370.0:0.030, 380.0:0.134, 390.0:0.567, 400.0:0.920,
         410.0:0.978, 420.0:1.000, 430.0:0.978, 440.0:0.935, 450.0:0.853,
         460.0:0.740, 470.0:0.640, 480.0:0.536, 490.0:0.424, 500.0:0.325,
         510.0:0.235, 520.0:0.150, 530.0:0.095, 540.0:0.043, 550.0:0.009,
         560.0:0.000}

    V = {470.0:0.000, 480.0:0.030, 490.0:0.163, 500.0:0.458, 510.0:0.780,
         520.0:0.967, 530.0:1.000, 540.0:0.973, 550.0:0.898, 560.0:0.792,
         570.0:0.684, 580.0:0.574, 590.0:0.461, 600.0:0.359, 610.0:0.270,
         620.0:0.197, 630.0:0.135, 640.0:0.081, 650.0:0.045, 660.0:0.025,
         670.0:0.017, 680.0:0.013, 690.0:0.009, 700.0:0.000}

    R = {550.0:0.00, 560.0:0.23, 570.0:0.74, 580.0:0.91, 590.0:0.98, 600.0:1.00,
         610.0:0.98, 620.0:0.96, 630.0:0.93, 640.0:0.90, 650.0:0.86, 660.0:0.81,
         670.0:0.78, 680.0:0.72, 690.0:0.67, 700.0:0.61, 710.0:0.56, 720.0:0.51,
         730.0:0.46, 740.0:0.40, 750.0:0.35, 800.0:0.14, 850.0:0.03, 900.0:0.00}

    I = {700.0:0.000, 710.0:0.024, 720.0:0.232, 730.0:0.555, 740.0:0.785,
         750.0:0.910, 760.0:0.965, 770.0:0.985, 780.0:0.990, 790.0:0.995,
         800.0:1.000, 810.0:1.000, 820.0:0.990, 830.0:0.980, 840.0:0.950,
         850.0:0.910, 860.0:0.860, 870.0:0.750, 880.0:0.560, 890.0:0.330,
         900.0:0.150, 910.0:0.030, 920.0:0.000}

    # Get wavelengths and tranmission values as arrays (for better interpolation)
    U_wavelength = np.array([k for k in U.keys()])/1e9
    B_wavelength = np.array([k for k in B.keys()])/1e9
    V_wavelength = np.array([k for k in V.keys()])/1e9
    R_wavelength = np.array([k for k in R.keys()])/1e9
    I_wavelength = np.array([k for k in I.keys()])/1e9
    U_trans = np.array([v for v in U.values()])
    B_trans = np.array([v for v in B.values()])
    V_trans = np.array([v for v in V.values()])
    R_trans = np.array([v for v in R.values()])
    I_trans = np.array([v for v in I.values()])

    # Create functions using interpolation to allow filter tranmittance from wavelength (in meters)
    # [any attempt to go outside the bounds gives a zero for the filter tranmission] 
    Uspline = interp1d(U_wavelength, U_trans, kind='cubic', bounds_error=False, fill_value = (0,0))
    Bspline = interp1d(B_wavelength, B_trans, kind='cubic', bounds_error=False, fill_value = (0,0))
    Vspline = interp1d(V_wavelength, V_trans, kind='cubic', bounds_error=False, fill_value = (0,0))
    Rspline = interp1d(R_wavelength, R_trans, kind='cubic', bounds_error=False, fill_value = (0,0))
    Ispline = interp1d(I_wavelength, I_trans, kind='cubic', bounds_error=False, fill_value = (0,0))

    ## At this point, I have defined the 5 functions to allow estimation of the transmission at any 
    ## wavelength through any of the Cousin-Johnson filters as global variables.
    return


def compute_UBVRIspectra(wavelengths, spectrum, debug=False):
    global Uspline, Bspline, Vspline, Rspline, Ispline
    
    '''
    This function will compute U, B, V, R, and I spectra, that is given the raw spectrum, how much gets through the 
    individual filters, is used to plot the data.
    '''
    
    # Check if the spectrum is acceptable, and if so, check that interpolation functions are defined.
    if (np.isfinite(spectrum).all()):
        # Check that the tranmission functions are defined by checking if Uspline is defined
        try:
            Uspline(500)
        except NameError:
            create_functions(debug=debug)
    else:
        raise ValueError('compute_UBVRIspectra: Your spectrum contains inf, death!')

    # Compute spectra getting through each filter (these are flux curves at surface)
    U_spectrum = spectrum*Uspline(wavelengths)
    B_spectrum = spectrum*Bspline(wavelengths)
    V_spectrum = spectrum*Vspline(wavelengths)
    R_spectrum = spectrum*Rspline(wavelengths)
    I_spectrum = spectrum*Ispline(wavelengths)
    
    return (U_spectrum, B_spectrum, V_spectrum, R_spectrum, I_spectrum)


def compute_flux_and_mag(wavelengths, spectrum, debug=False):
    global Uspline, Bspline, Vspline, Rspline, Ispline, delta_V, delta_lambda
    global U_spec, B_spec, V_spec, R_spec, I_spec
    
    '''
    This function will compute U, B, V, R, and I magnitudes for the spectrum you pass it.  The magnitudes are UNCALIBRATED
    and so they should NOT be displayed to the user, however, they are on the same system, so they are appropriate for 
    color index calculation.  However, if you want to fake it, I set it up so they are all scaled so V = V_calbration.
    '''

    # Compute spectra getting through each filter
    (U_spec, B_spec, V_spec, R_spec, I_spec) = compute_UBVRIspectra(wavelengths, spectrum, debug=debug)
        
    # Compute the cumilative fluxes through each filter at the surface of the star
    U_flux = U_spec.sum()*delta_lambda
    B_flux = B_spec.sum()*delta_lambda
    V_flux = V_spec.sum()*delta_lambda
    R_flux = R_spec.sum()*delta_lambda
    I_flux = I_spec.sum()*delta_lambda
    
    # Convert flux for 1 solar radius star at 10 pc distance 
    scale_factor = (R_Sun/d_10pc)**2
    U_flux *= scale_factor
    B_flux *= scale_factor
    V_flux *= scale_factor
    R_flux *= scale_factor
    I_flux *= scale_factor
    
    # Uncalibrated magnitudes reset so V_mag value is set to V_calibration
    # Absolute magnitude of the Sun in UBVRI a la http://mips.as.arizona.edu/~cnaw/sun.html#filters
    abs_mag = np.array([6.33, 5.31, 4.80, 4.60, 4.51])
    # Compute raw magnitudes for Sun computed using T=5777 in this algorithm
    raw_mag = np.array([28.01, 27.30, 27.34, 26.87, 27.24])
    # Correction
    corr = raw_mag-abs_mag
    
    U_mag = -2.5*np.log10(U_flux) - corr[0]
    B_mag = -2.5*np.log10(B_flux) - corr[1]
    V_mag = -2.5*np.log10(V_flux) - corr[2]
    R_mag = -2.5*np.log10(R_flux) - corr[3]
    I_mag = -2.5*np.log10(I_flux) - corr[4]
    
    return(U_mag, B_mag, V_mag, R_mag, I_mag, U_flux, B_flux, V_flux, R_flux, I_flux)


def ConfigStars(r, t):
    '''
    Determines the radii (in solar radii), temperature (in K), and hexcolor of the two stars assuming 
    they are main sequence stars and returns that information.
    '''
    # Determine approximate radius in solar radii for both stars.
    radius1= r

    # Determines the approximate temperature of each star.
    temp1 = t
    
    # Use scalar temperature to estimate hexcolor appropriate to each star
    hexcolor1 = tc.rgb2hex(tc.temp2rgb(temp1))[0]
    
    return (radius1, temp1, hexcolor1)


def func(lam, A, T):
    """
    Returns a Planck curve scaled by the factor A
    """
    return A*blackbody(lam, T)


def get_spectral_data(index):
    """
    Takes the index value and returns numpy arrays of wavelength and flux for that object.
    """
    wl = np.array(spec_data['Wavelengths'][index].split(':')).astype(np.float64)
    flux = np.array(spec_data['Fluxes'][index].split(':')).astype(np.float64)
    return wl, flux


def get_title(index):
    return "Spectra of "+spec_data['Name'][index]+" (Spectral Type "+spec_data['Type'][index]+")"    


def EstTempAndRescale(lam, flux, sigma_est):
    '''
    This does a piss poor approximation of the temperature by getting the peak wavelength (using a simple maximum)
    and then assuming Wien's Law.
    '''
    
    # Assume all points have same uncertainty
    sig = sigma_est*np.ones(len(lam))
    
    # Find the wavelength of maximum flux and then determine its temperature assuming Wein's Law
    lamb_max = lam[flux.argmax()]
    T = 0.002897755 / lamb_max
    
    # Get the Planck Curve
    planck = blackbody(lam, T)
    # Estimate scaling factor
    A= flux.max()/planck.max()
    
    # Attempt a more robust fit using SciPy's curve fit
    popt, pcov = curve_fit(func, lam, flux, p0=(A, T), sigma=sig)
    
    # get the best fitting parameter values and their 1 sigma errors
    bestA, bestT = popt
    sigmaA, sigmaT = np.sqrt(np.diag(pcov))
    
    # Rescale fluxes and smoothed fluxes
    flux /= bestA
    
    # Enter temperature and peak wavelength
    bestTemp = round(bestT, -2)
    temp_text.value = str(int(bestTemp))
    scale_text.value = str(1/bestA)
    
    # Return the best fit Planck Spectrum
    return (flux, blackbody(lam, bestT))

In [ ]:
# Variable Definition Block
# Open the necessary datafile
spec_data = pd.read_csv('data/spec_data_use.csv')

# Define the peak wavelength for initial calculations and plots
pw = Wein(2800) 

# Define all wavelengths from 1 nm to 1600 nm in steps of 1nm (use float128 to avoid overflows)
delta_lambda = 1e-9
wavelengths = np.arange(1.0e-9, 1601.0e-9, delta_lambda, dtype=np.float128)
wavelengths_m = wavelengths/(1e9)

pwlist = nf.exp2LaTeX(pw) # Make a list from the number2LaTeX converter being used

spectrum = blackbody(wavelengths, 2800) # Generate an initial spectrum to be used in the early calculations

# Initialize filter curves here
(U_spec, B_spec, V_spec, R_spec, I_spec) = compute_UBVRIspectra(wavelengths, spectrum, debug=False)

# Calculate the initial U, B, V, R, I magnitudes and spectra, activate debug to plot for early visual plot otherwise
#    values will be plotted later
(U_mag, B_mag, V_mag, R_mag, I_mag, U_flux, B_flux, V_flux, R_flux, I_flux) = compute_flux_and_mag(wavelengths, spectrum, debug=False)

# Get the values for the color
UB_color = U_mag - B_mag
BV_color = B_mag - V_mag
VR_color = B_mag - R_mag
RI_color = R_mag - I_mag

In [ ]:
# Widget Definitions Block
# This widget controls the temperature of the black body
Temp = widgets.FloatSlider(
    min=2800,
    max=65000,
    step=100,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

# This widget controls the temperature of the black body in the 2nd interactive
Temp2 = widgets.FloatSlider(
    min=2800,
    max=65000,
    step=100,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

# This widget controls the temperature of the black body in the 3rd interactive
Temp3 = widgets.FloatSlider(
    min=2800,
    max=65000,
    step=100,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

# Widget to report updated peak wavelength 
peak_wavelength = widgets.HTML(
    value = '{}'.format(pwlist[2]),
    disabled = True   
)

# Update all the color indicies 
UB_color_report = widgets.Label(
    value = "{0:.2f}".format(UB_color),
    disabled = True   
)

BV_color_report = widgets.Label(
    value = "{0:.2f}".format(BV_color),
    disabled = True   
)

VR_color_report = widgets.Label(
    value = "{0:.2f}".format(VR_color),
    disabled = True   
)

RI_color_report = widgets.Label(
    value = "{0:.2f}".format(RI_color),
    disabled = True   
)

# This allows us to chose between stars
plotted_star = widgets.RadioButtons(
    options=['Proxima Centauri', 'Sun', 'Polaris', 'Alpha Andromedae', 'Eta Ursae Majoris', 'Bellatrix'],
    disabled=False
)

# These widgets report the U, B, V, R, I fluxes and magnitudes
U_mag_report = widgets.HTML(
    value = "{0:.2f}".format(U_mag),
    disabled = True   
)

B_mag_report = widgets.HTML(
    value = "{0:.2f}".format(B_mag),
    disabled = True   
)

V_mag_report = widgets.HTML(
    value = "{0:.2f}".format(V_mag),
    disabled = True   
)

R_mag_report = widgets.HTML(
    value = "{0:.2f}".format(R_mag),
    disabled = True   
)

I_mag_report = widgets.HTML(
    value = "{0:.2f}".format(I_mag),
    disabled = True   
)

U_flux_report = widgets.HTML(
    value = str('{}'.format(nf.exp2LaTeX(U_flux,3)[2])),
    disabled = True   
)

B_flux_report = widgets.HTML(
    value = str('{}'.format(nf.exp2LaTeX(B_flux,3)[2])),
    disabled = True   
)

V_flux_report = widgets.HTML(
    value = str('{}'.format(nf.exp2LaTeX(V_flux,3)[2])),
    disabled = True   
)

R_flux_report = widgets.HTML(
    value = str('{}'.format(nf.exp2LaTeX(R_flux,3)[2])),
    disabled = True   
)

I_flux_report = widgets.HTML(
    value = str('{}'.format(nf.exp2LaTeX(I_flux,3)[2])),
    disabled = True   
)

# Build a widget to select star to display
starlist = widgets.Dropdown(
    options=list(spec_data['Name']),
    value=spec_data['Name'][0],
    description='Star Name:',
    style = {'description_width': 'initial'},
    disabled=False
)

ind_text = widgets.Text(
    value=str(spec_data[spec_data['Name'] == starlist.value].index[0]),
    description='Index:',
    style = {'description_width': 'initial'},
    disabled=False
)

temp_text = widgets.Text(
    value=str(0),
    description='Planck Temp (K):',
    style = {'description_width': 'initial'},
    disabled=True
)

scale_text = widgets.Text(
    value=str(0),
    description='Scale:',
    style = {'description_width': 'initial'},
    disabled=True
)

planck_temp = widgets.ToggleButton(
    value=False,
    description='Show Estimated Temperature',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)

In [ ]:
# Required Data Block
# Update peak wavelength
my_f = blackbody(wavelengths, Temp.value)

In [ ]:
# Figure 1 Definition Block
# Define the scale
x_sc1 = bq.LinearScale()
y_sc1 = bq.LinearScale()

# Define the axes
ax_x = bq.Axis(label='Wavelength (m)', scale=x_sc1, tick_format='0.0e')
ax_y = bq.Axis(label='Intensity', scale=y_sc1, orientation='vertical', tick_format='0.2g')
ax_y.label_offset = '6ex'

# Convert wavelengths to plottable unis (bqplot is unhappy with np.float128)
wavelengths2plot = np.array(wavelengths, dtype=float)

# Define the markers which are the blackbody and the filter curves
Blackbody = bq.Lines(x=wavelengths2plot, 
             y=spectrum,
             name = ['Model Blackbody'],
             scales={'x': x_sc1, 'y': y_sc1},
             display_legend=True,
             colors=['orange'], 
             labels=['Model Star'])

U_line = bq.Lines(x=wavelengths2plot,
             y=U_spec,
             name = ['U Filter'],
             scales={'x': x_sc1, 'y': y_sc1},
             display_legend=True,
             fill = 'inside',
             fill_colors = ['violet'],
             opacities = [0.5],
             colors=['violet'], 
             labels=['U Filter'])

B_line = bq.Lines(x=wavelengths2plot,
             y=B_spec,
             name = ['B Filter'],
             scales={'x': x_sc1, 'y': y_sc1},
             display_legend=True,
             fill = 'inside',
             fill_colors = ['blue'],
             opacities = [0.5],
             colors=['blue'], 
             labels=['B Filter'])

V_line = bq.Lines(x=wavelengths2plot, 
             y=V_spec,
             name = ['V Filter'],
             scales={'x': x_sc1, 'y': y_sc1},
             display_legend=True,
             fill = 'inside',
             fill_colors = ['green'],
             opacities = [0.5],
             colors=['green'], 
             labels=['V Filter'])

R_line = bq.Lines(x=wavelengths2plot, 
             y=R_spec,
             name = ['R Filter'],
             scales={'x': x_sc1, 'y': y_sc1},
             display_legend=True,
             fill = 'inside',
             fill_colors = ['red'],
             opacities = [0.5],
             colors=['red'], 
             labels=['R Filter'])

I_line = bq.Lines(x=wavelengths2plot,
             y=I_spec,
             name = ['I Filter'],
             scales={'x': x_sc1, 'y': y_sc1},
             display_legend=True,
             fill = 'inside',
             fill_colors = ['black'],
             opacities = [0.5],
             colors=['black'], 
             labels=['I Filter'])

# Call the function to update the figure in real time
Temp.observe(cr, names=['value'])

# Define Figure 1
fig = bq.Figure(title = 'Blackbody with Color Filters',axes=[ax_x, ax_y], animation = 1000, 
                marks=[Blackbody, U_line, B_line, V_line, R_line, I_line],
                legend_location='top-left')

In [ ]:
# Figure 2 Definition Block
# Define the scale
x_sc2 = bq.LinearScale()
y_sc2 = bq.LinearScale()

# Define the axes
ax_x2 = bq.Axis(label='Wavelength (m)', scale=x_sc1, tick_format='0.0e')
ax_y2 = bq.Axis(label='Intensity', scale=y_sc1, orientation='vertical', tick_format='0.2g')
ax_y2.label_offset = '6ex'

# This code define the colors to be plotted and which wavelengths they are plotted at, uses hexidecimal designation
colors_array = tc.wav2hex(wavelengths*1e9)
colors_list = colors_array.tolist()

# The curve for this figure is created by drawing a bunch of vertical lines that go from zero to the blackbody curve, these 
#     arrays provide the proper pairs of points that define each individual line
x_array = np.array([wavelengths2plot, wavelengths2plot])
fin_x_array = x_array.transpose() # Arrays must be transposed to get pairs of numbers
y_zeros = np.zeros_like(wavelengths2plot)
y_array = np.array([y_zeros, my_f])
fin_y_array = y_array.transpose() # Arrays must be transposed to get pairs of numbers

# This is the line command that draws all the lines
wide_line = bq.Lines(x = fin_x_array, y = fin_y_array, scales={'x': x_sc2, 'y': y_sc2}, colors=colors_list)

# Implement the update function
Temp.observe(fini, names=['value'])

# Define Figure 2
fig2 = bq.Figure(title = 'Model Blackbody Spectrum', axes=[ax_x2, ax_y2], animation = 1000, marks = [wide_line])

In [ ]:
# Interactive Figure 2 Stellar Image Definition Block
# Set scale factor for radius (10 pixels per solar radius)
scale_factor = 1

# Set viewer size
view_width = 400
view_height = 400

# Set initial parameters based on stellar parameters
(radius1, temp1, hexcolor1) = ConfigStars(5, Temp.value)
r1 = scale_factor*radius1

# Save initial radius to scale all other radii to this
init_r = r1

# set the scale
scale1 = (r1/init_r, r1/init_r, r1/init_r)

# Create a star using StarMesh
star1 = star.StarMesh(temp1, radius1, scale1)

# Define viewing region size
xmax=1.5*10

# Makes the scene environment, not sure how the background works yet
scene2 = p3j.Scene(children=[star1], background='black')

# Creates the camera so you can see stuff.  Place the cemera just above the x-axis and orient camera so up
# is along y-axis.
starcam = p3j.PerspectiveCamera(position=[1.25*xmax, 0.1*xmax, 0], up=[0, 1, 0])

# Makes a controller to use for the 
controller = p3j.OrbitControls(controlling=starcam, enableRotate=True, enableZoom=False)

# creates the object that gets displayed to the screen
renderer2 = p3j.Renderer(camera=starcam, 
                    scene=scene2, 
                    controls=[controller],
                    width=view_width, height=view_height)

box_layout = widgets.Layout(align_items='center', justify_content = 'flex-end', border='none', width='100%')

fig3 = widgets.VBox([widgets.HTML ("<h3>Blackbody Star Appearance</h3>"), renderer2], layout = box_layout)

In [ ]:
# star_fig Definition Block

# Define the scale
x_star = bq.LinearScale()
y_star = bq.LinearScale(max = float(blackbody(Wein(3300), 3300)))

# Define the axes
ax_x_star = bq.Axis(label='Wavelength (m)', scale=x_star, tick_format='0.0e', grid_color = 'black')
ax_y_star = bq.Axis(label='Intensity', scale=y_star, orientation='vertical', tick_format='0.2g', 
                    grid_color = 'black')
ax_y_star.label_offset = '6ex'

# Define the Markers
Sun = bq.Lines(x = wavelengths2plot,
             y = blackbody(wavelengths,5800),
             name = ['5800 K'],
             display_legend=True,
             scales={'x': x_star, 'y': y_star},
             colors=[tc.rgb2hex(tc.temp2rgb(5800))[0]],
             labels=['The Sun'])


Proxima_Centauri = bq.Lines(x = wavelengths2plot,
             y = blackbody(wavelengths,3300),
             name = ['3300 K'],
             display_legend=True,
             scales={'x': x_star, 'y': y_star},
             colors=[tc.rgb2hex(tc.temp2rgb(3300))[0]],
             labels=['Proxima Centauri'])


Polaris = bq.Lines(x = wavelengths2plot,
             y = blackbody(wavelengths,7200),
             name = ['7200 K'],
             display_legend=True,
             scales={'x': x_star, 'y': y_star},
             colors=[tc.rgb2hex(tc.temp2rgb(7200))[0]],
             labels=['Polaris'])

Alpha_Andromedae = bq.Lines(x = wavelengths2plot,
             y = blackbody(wavelengths,13000),
             name = ['13000 K'],
             display_legend=True,
             scales={'x': x_star, 'y': y_star},
             colors=[tc.rgb2hex(tc.temp2rgb(13000))[0]],
             labels=['Alpha Andromedae'])

Eta_Ursae_Majoris = bq.Lines(x = wavelengths2plot,
             y = blackbody(wavelengths, 16900),
             name = ['16900 K'],
             display_legend=True,
             scales={'x': x_star, 'y': y_star},
             colors=[tc.rgb2hex(tc.temp2rgb(16900))[0]],
             labels=['Eta Ursae Majoris'])

Bellatrix = bq.Lines(x = wavelengths2plot,
             y = blackbody(wavelengths,22000),
             name = ['22000 K'],
             display_legend=True,
             scales={'x': x_star, 'y': y_star},
             colors=[tc.rgb2hex(tc.temp2rgb(22000))[0]],
             labels=['Bellatrix'])

Model_Star = bq.Lines(x = wavelengths2plot,
             y = blackbody(wavelengths,2800),
             name = ['Model'],
             display_legend=True,
             scales={'x': x_star, 'y': y_star},
             colors=['#ff00ff'],
             labels=['Model Star'])

# Call the function to update the figure in real time
Temp.observe(cr, names=['value'])
plotted_star.observe(cr, names=['index'])

# Define star_fig
star_fig = bq.Figure(title = 'Model Stellar Spectra', axes=[ax_x_star, ax_y_star], animation = 1000, 
                marks=[Proxima_Centauri, Model_Star], background_style ={'fill':'black'}, 
                legend_location='top-left')

In [ ]:
# Interactive Figure 3 definition block
# Parameters for Savitzky-Golay filter
window = 21
order = 3

# Define the axes (and their scales)
x_sc3 = bq.LinearScale()
y_sc3 = bq.LinearScale()
ax_x3 = bq.Axis(label='Wavelength (m)', scale=x_sc3)
ax_y3 = bq.Axis(label='Intensity', scale=y_sc3, orientation='vertical')

# Get the data
index = int(ind_text.value)
(lam, flux) = get_spectral_data(index)
title_text = get_title(index)

# Smooth the data by applying a Savitzky-Golay filter (just a nice available filter for 
# noisy 1D data) and use this to estimate uncertainty in individual measurements.
sigma = 2*np.std(flux-savgol_filter(flux, window, order))

# Try to fit the Planck spectrum and estimate temperature, then rescale the flux to match
(flux, planck) = EstTempAndRescale(lam, flux, sigma)

# Create figure
spectra = bq.Lines(x=lam, y=flux, 
                   scales={'x': x_sc3, 'y': y_sc3}, 
                   colors =['blue'], 
                   display_legend=True,
                   labels=['Raw Data'])

planck_spectra = bq.Lines(x=lam, y=planck, 
                            scales={'x': x_sc3, 'y': y_sc3}, 
                            colors =['orange'], 
                            display_legend=True,
                            labels=['Planck (Estimated)'])

Model_Star_Vis = bq.Lines(x = lam,
                          y = blackbody(lam,5800),
                          name = ['Model'],
                          display_legend=True,
                          scales={'x': x_sc3, 'y': y_sc3},
                          colors=['black'],
                          labels=['Model Star'])

fig4 = bq.Figure(title=title_text, marks=[spectra, Model_Star_Vis], axes=[ax_x3, ax_y3], 
                legend_location='top-right')
fig4.layout.width = '700px'

## Interactive Figure 1: A Blackbody Spectrum Model

The figure on the left shows the spectrum of a model star and several other known stars.  It assumes the stars are perfect blackbodies and give off is a "Blackbody Spectrum" (also called a "Planck", "Thermal" or "Continuous" Spectrum).  Adjust the temperature slider until you have matched the spectra of the star with that of the blackbody model and you will have an estimate for the surface temperature of the star.

The figure on the right also shows the same spectrum, highlighting the visible potion of the spectrum. It allows you to visually see at what colors the spectrum peaks as the temperature of the blackbody changes.

In [ ]:
#star_fig
temp_slide = widgets.HBox([widgets.HTML('<B>Temperature (K)</B>:'),Temp])
star_selector = widgets.HBox([widgets.HTML('<B>Stars:</B>'),plotted_star])
BOX = widgets.HBox([star_fig, fig2], layout = widgets.Layout(align_items = 'center'))
BOX.children[0].layout.width = '450px' # Resize Figure 1 so that both figures fit on the screen
BOX.children[1].layout.width = '450px' # Resize Figure 2 so that both figures fit on the screen
selectors = widgets.HBox([star_selector, temp_slide], layout = widgets.Layout(align_items = 'center'))
fig_box = widgets.VBox([BOX, selectors], layout = widgets.Layout(align_items = 'center'))

display(fig_box)

## Interactive Figure 2: Temperature vs. Apparent Color of Stars

As you can see using Interactive Figure 1, the temperature of a blackbody (and hopefully a star) is strongly tied to the peak wavelength of light it emits.  However, the "peak wavelength" is not the only color of light given off.  A blackbody gives off light of many colors, what we see is a mixture of the light emitted across the entire visible spectrum.  **Note**: This interactive was inspired by this [Flash-based UNL Blackbody Curves activity](http://astro.unl.edu/classaction/animations/light/bbexplorer.html).
   
Now examine the interactive figure below.  

- The graph to the left below show how much flux (brightness) the blackbody gives off in each color filter range (labelled U, B, V, R, and I). You can use to see how the differences in the amount of light in two of these filters can be used to quantify the color of the star.  
- The image on the right shows you the color of the star as you would see it if you looked at it.  Try adjusting the slider.  Which colors are hotter and which are cooler?

In [ ]:
# Display Block
# Label the all the widgets to prevent text cut-off
p_wave = widgets.HBox([widgets.HTML('<B>Peak Wavelength (m)</B>:'), peak_wavelength])
p_wave.children[1].layout.width = '100px' # Make this widget large enough to handle the LaTeX notation

ufl =widgets.HBox([widgets.HTML('<B>U Flux (W/m<sup>2</sup>)</B>:'), U_flux_report])
ufl.children[1].layout.width = '85px'
bfl =widgets.HBox([widgets.HTML('<B>B Flux (W/m<sup>2</sup>)</B>:'), B_flux_report])
bfl.children[1].layout.width = '85px'
vfl =widgets.HBox([widgets.HTML('<B>V Flux (W/m<sup>2</sup>)</B>:'), V_flux_report])
vfl.children[1].layout.width = '85px'
rfl =widgets.HBox([widgets.HTML('<B>R Flux (W/m<sup>2</sup>)</B>:'), R_flux_report])
rfl.children[1].layout.width = '85px'
ifl =widgets.HBox([widgets.HTML('<B>I Flux (W/m<sup>2</sup>)</B>:'), I_flux_report])
ifl.children[1].layout.width = '85px'

umagcol =widgets.HBox([widgets.HTML('<B>U</B>:'), U_mag_report])
umagcol.children[1].layout.width = '100px'
bmagcol =widgets.HBox([widgets.HTML('<B>B</B>:'), B_mag_report])
bmagcol.children[1].layout.width = '100px'
vmagcol =widgets.HBox([widgets.HTML('<B>V</B>:'), V_mag_report])
vmagcol.children[1].layout.width = '100px'
rmagcol =widgets.HBox([widgets.HTML('<B>R</B>:'), R_mag_report])
rmagcol.children[1].layout.width = '100px'
imagcol =widgets.HBox([widgets.HTML('<B>I</B>:'), I_mag_report])
imagcol.children[1].layout.width = '100px'

ubcol =widgets.HBox([widgets.HTML('<B>U-B Color Index</B>:'), UB_color_report])
ubcol.children[1].layout.width = '100px'
bvcol =widgets.HBox([widgets.HTML('<B>B-V Color Index</B>:'), BV_color_report])
bvcol.children[1].layout.width = '100px'
vrcol =widgets.HBox([widgets.HTML('<B>V-R Color Index</B>:'), VR_color_report])
vrcol.children[1].layout.width = '100px'
ricol =widgets.HBox([widgets.HTML('<B>R-I Color Index</B>:'), RI_color_report])
ricol.children[1].layout.width = '100px'

# Make the figures the focus of the screen
top = widgets.HBox([fig, fig3]) # Make the visual figures appear on the top for eay veiwing
top.children[0].layout.width = '450px' # Resize Figure 1 so that both figures fit on the screen
top.children[1].layout.width = '450px' # Resize Figure 2 so that both figures fit on the screen
top.children[1].layout.height = '450px' # Resize Figure 2 so that both figures fit on the screen

# Make the magnitude display widgets stand vertically on the side
upper_middle = widgets.HBox([ufl, bfl, vfl, rfl,ifl])
middle_top = widgets.HBox([umagcol, bmagcol, vmagcol, rmagcol, imagcol])
middle = widgets.HBox([ubcol, bvcol, vrcol, ricol])
middle_alt = widgets.HBox([vfl, bfl, bvcol])

# Assign peak wavelength and the temperature slider to sit under the the figures
temp2_slide = widgets.HBox([widgets.HTML('<B>Temperature (K)</B>:'),Temp2])
bottom = widgets.HBox([temp2_slide, p_wave])

# Link slider to function to update display
Temp2.observe(cr2, names=['value'])

# Organize the widgets to display in a presentable fashion 
if (MSUM):
    box = widgets.VBox([top, middle_alt, bottom], layout = widgets.Layout(align_items = 'center'))
else:
    box = widgets.VBox([top, upper_middle, middle_top, middle, bottom], layout = widgets.Layout(align_items = 'center'))
    
big_box = widgets.HBox([box], layout = widgets.Layout(align_items = 'center'))
display(big_box) # Display it

## Interactive Figure 3: Real Stellar Spectra

The following plot shows real spectra of stars.  Though we model stars as blackbodies, real stellar spectra show they are not perfect blackbodies.  The real spectra of stars depend on a variety of factors beyond temperature.  Some of these factors include the chemical composition of the star, the temperature dependence of the chemical's interactions with light, and so on.  Despite the complications, the broad shape or "continuous" part of a stellar spectrum can typically be fit with a blackbody spectrum.  Use the temperature slider to estimate the temperature of the real stars and then click the 'Show Estimated Temperature' button to see how close you were.  You can select a new star, and try this again. 

In [ ]:
# Display everything
temp_slide3 = widgets.HBox([widgets.HTML('<B>Temperature (K)</B>:'), Temp3])

full_top = widgets.HBox([starlist])
full_mid = widgets.HBox([temp_slide3, planck_temp])
full_mid.children[1].layout.width = '250px'
full_bottom = fig4
full_window = widgets.VBox([full_top, full_mid, full_bottom], layout = widgets.Layout(align_items = 'center'))

# This line is here for the authors needs
#full_window = widgets.VBox([widgets.HBox([starlist, ind_text]), widgets.HBox([scale_text, temp_text]), temp_slide, fig4])

display(full_window)

# React to changes in star choosen
starlist.observe(new_star, 'value')
Temp3.observe(new_blackbody, 'value')
planck_temp.observe(new_blackbody, 'value')